<a href="https://colab.research.google.com/github/Meer2001/RAG-Workshop/blob/main/Copy_of_rag_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Workshop with LangGraph and Gemini API

## Access the tinyurl link to open the colab and make a copy of it to follow along!

File > Save a Copy in Drive

### https://tinyurl.com/simple-rag-ais

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) system using:
- **LangGraph** for RAG pipeline orchestration
- **Gemini API** for text generation
- **PDF documents** from a data directory
- **Vector embeddings** for semantic search
- **ChromaDB** for vector storage and retrieval


In [ ]:
# Install required packages (run this cell if packages are not installed)
!pip install -q langchain langgraph langchain-google-genai langchain-community langchain-core langchain-text-splitters python-dotenv pypdf chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 

Use the following api key: https://pastebin.com/NJrVX2xG. To create a secret in google colab:
1. Click on the key icon in the sidebar of the notebook
2. Add new secret
3. Type in gemini for the name and the value.
4. Enable notebook access for the secret

Note: The api key will be destroyed after this workshop, if you'd like to play with these scripts later, visit: https://aistudio.google.com for your own api key!

In [ ]:
# Import required libraries
import os
from typing import TypedDict, List
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
import glob
from google.colab import userdata

# Load environment variables

# Verify API key is loaded
GEMINI_API_KEY = userdata.get('gemini')
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in .env file")

# Also set as GOOGLE_API_KEY for LangChain compatibility
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

print("Environment variables loaded")


Environment variables loaded


## Step 1: Load PDF Documents

This step prepares the document collection for the RAG system:

- **Download documents from Google Drive**: Authenticate with Google Drive API and copy the shared folder containing PDF documents to your Drive
- **Locate PDF files**: Search for all PDF files in the specified data directory
- **Load PDF documents**: Use PyPDFLoader to extract text content from each PDF file
- **Add metadata**: Attach source file information to each document for traceability
- **Display summary**: Show the total number of documents loaded and total character count across all pages

Replace the text at ```shared_link = "INSERT LINK HERE"``` with "https://drive.google.com/drive/folders/1iwWlvtMj-QY0usHFQGqPGDv1Vu9Y7yaq". This is how we copy the 'data' folder from someone else's drive into our own.

Make sure you allow all permissions for your drive during the mounting authentication for this to work.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q pydrive2

from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.discovery import build
import re

auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
service = build("drive", "v3", credentials=gauth.credentials)

def extract_id_from_url(url: str) -> str:
    patterns = [
        r"/d/([a-zA-Z0-9_-]+)",
        r"id=([a-zA-Z0-9_-]+)",
        r"/folders/([a-zA-Z0-9_-]+)",
    ]
    for p in patterns:
        m = re.search(p, url)
        if m:
            return m.group(1)
    return url.strip()

def copy_file(service, file_id, parent_id=None, new_name=None):
    body = {}
    if new_name:
        body["name"] = new_name
    if parent_id:
        body["parents"] = [parent_id]

    copied = service.files().copy(
        fileId=file_id,
        body=body,
        fields="id, name"
    ).execute()
    return copied

def create_folder(service, name, parent_id=None):
    metadata = {
        "name": name,
        "mimeType": "application/vnd.google-apps.folder"
    }
    if parent_id:
        metadata["parents"] = [parent_id]

    folder = service.files().create(
        body=metadata,
        fields="id, name"
    ).execute()
    return folder

def copy_folder_recursive(service, source_folder_id, target_parent_id=None):
    src = service.files().get(
        fileId=source_folder_id,
        fields="id, name"
    ).execute()

    new_folder = create_folder(
      service,
      name=src["name"],
      parent_id=target_parent_id
    )
    new_folder_id = new_folder["id"]

    page_token = None
    while True:
        response = service.files().list(
            q=f"'{source_folder_id}' in parents and trashed=false",
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token
        ).execute()

        for item in response.get("files", []):
            if item["mimeType"] == "application/vnd.google-apps.folder":
                copy_folder_recursive(service, item["id"], new_folder_id)
            else:
                copy_file(
                    service,
                    file_id=item["id"],
                    parent_id=new_folder_id,
                    new_name=item["name"]
                )

        page_token = response.get("nextPageToken", None)
        if page_token is None:
            break

    return new_folder

shared_link = "https://drive.google.com/drive/folders/1iwWlvtMj-QY0usHFQGqPGDv1Vu9Y7yaq"

source_id = extract_id_from_url(shared_link)

meta = service.files().get(
    fileId=source_id,
    fields="id, name, mimeType"
).execute()

mime = meta["mimeType"]
name = meta["name"]

if mime == "application/vnd.google-apps.folder":
    new_folder = copy_folder_recursive(service, source_id, target_parent_id=None)
    new_link = f"https://drive.google.com/drive/folders/{new_folder['id']}"
    print(f"Copied folder: {new_link}")
else:
    copied = copy_file(
        service,
        file_id=source_id,
        parent_id=None,
        new_name=name
    )
    new_link = f"https://drive.google.com/file/d/{copied['id']}/view"
    print(f"Copied file: {new_link} into your drive")

Copied folder: https://drive.google.com/drive/folders/1stoaQI0b1QqDFHOUX0PLL-JvoNizsjFl


In [ ]:
# Load all PDF documents from the data directory (that is now inside of your drive)
pdf_files = glob.glob(os.path.join("./drive/MyDrive/data", "*.pdf"))

if not pdf_files:
    print("No PDF files found in data directory")
    documents = []
else:
    documents = []
    for pdf_path in pdf_files:
        try:
            loader = PyPDFLoader(pdf_path)
            docs = loader.load()
            # Add source metadata to each document
            for doc in docs:
                doc.metadata['source'] = os.path.basename(pdf_path)
            documents.extend(docs)
            print(f"Loaded: {os.path.basename(pdf_path)} ({len(docs)} pages)")
        except Exception as e:
            print(f"Error loading {pdf_path}: {e}")

print(f"\nTotal documents loaded: {len(documents)}")
print(f"Total pages: {sum([len(doc.page_content) for doc in documents])} characters")


Loaded: Ch 10 Chisquare_Test.pdf (13 pages)
Loaded: Ch 9 Hypothesis Test.pdf (30 pages)
Loaded: Ch 5 Simulation.pdf (12 pages)
Loaded: Chapter 1-4 Review of Prob.pdf (121 pages)
Loaded: Probability and statistics for computer scientists.pdf (487 pages)
Loaded: Ch 6-graphical_statistics.pdf (17 pages)
Loaded: Ch 7-point_estimation.pdf (14 pages)
Loaded: Ch 8 Confidence_Intervals.pdf (43 pages)

Total documents loaded: 737
Total pages: 1019317 characters


## Step 2: Chunk Documents

This step breaks down large documents into smaller, manageable pieces for efficient processing:

- **Initialize text splitter**: Configure RecursiveCharacterTextSplitter with optimal chunk size (1000 characters) and overlap (200 characters) to maintain context between chunks
- **Split documents**: Process all loaded documents and divide them into smaller text chunks
- **Preserve structure**: Use separators (paragraphs, sentences, words) to split at natural boundaries
- **Maintain metadata**: Keep source information attached to each chunk for proper attribution
- **Display statistics**: Show total number of chunks created and average chunk size for verification

Follow along with the workshop and replace the ```"INSERT"``` quotes with actual code.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

texts = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(texts)}")
print(f"Average chunk size: {sum([len(chunk.page_content) for chunk in texts]) / len(texts):.0f} characters")
print(f"\nSample chunk:")
print(f"Source: {texts[0].metadata.get('source', 'Unknown')}")
print(f"Content preview: {texts[0].page_content[:200]}...")


NameError: name 'documents' is not defined

## Step 3: Create Vector Store with ChromaDB

This step converts text chunks into vector embeddings and stores them for semantic search:

- **Initialize embeddings model**: Set up Google's embedding model (embedding-001) to convert text into high-dimensional vectors
- **Generate embeddings**: Transform all document chunks into vector representations that capture semantic meaning
- **Create vector database**: Use ChromaDB to store embeddings with their associated text chunks and metadata
- **Persist to disk**: Save the vector store locally so it can be reused without regenerating embeddings
- **Verify creation**: Confirm the vector store contains all document chunks and is ready for retrieval

Follow along with the workshop and replace the ```"INSERT"``` quotes with actual code.

In [ ]:

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GEMINI_API_KEY
)

print("Creating vector store with ChromaDB...")

vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

# Load existing vector store (uncomment to use instead of creating new)
# vectorstore = Chroma(
#     persist_directory="./chroma_db",
#     embedding_function=embeddings
# )

print(f"Vector store created with {len(texts)} documents")
print(f"Database persisted to: ./chroma_db")


Creating vector store with ChromaDB...
Vector store created with 1528 documents
Database persisted to: ./chroma_db


## Step 4: Initialize Gemini LLM

This step sets up the language model that will generate answers based on retrieved context:

- **Configure Gemini model**: Initialize the ChatGoogleGenerativeAI client with the gemini-flash-latest model
- **Set temperature**: Configure temperature (0-1) to balance between focused and creative responses
- **Handle system messages**: Enable conversion of system messages to human messages for compatibility
- **Authenticate**: Provide API key for secure access to Google's Gemini API
- **Verify initialization**: Confirm the LLM is ready to process queries and generate responses

Follow along with the workshop and replace the ```"INSERT"``` quotes with actual code.


In [ ]:
# Initialize Gemini LLM
# Explicitly pass the API key to avoid using Application Default Credentials
# gemini-flash-latest model
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest",  # Updated to current model name
    temperature=1,
    convert_system_message_to_human=True,
    google_api_key=GEMINI_API_KEY
)

print("Gemini LLM initialized")


Gemini LLM initialized


## Step 5: Create RAG Graph with LangGraph

This step assembles the complete RAG pipeline as an explicit graph using LangGraph:

- **Design prompt template**: Create a comprehensive prompt that instructs the LLM to answer questions using retrieved context while allowing for interpretation and synthesis
- **Define state schema**: Create a `RAGState` TypedDict with `question`, `context`, and `answer` fields that flow through the graph
- **Configure retriever**: Set up the vector store retriever to fetch the top 3 most relevant document chunks for each query
- **Create node functions**: Define a `retrieve` node (fetches relevant docs from the vector store) and a `generate` node (formats context, prompts the LLM, and extracts the answer)
- **Build the graph**: Use LangGraph's `StateGraph` to wire nodes together with edges: `retrieve` -> `generate` -> `END`
- **Compile the graph**: Call `workflow.compile()` to produce a runnable graph that processes queries end-to-end

Follow along with the workshop and replace the ```"INSERT"``` quotes with actual code.

In [ ]:
import os
from typing import TypedDict, List
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
import glob
from google.colab import userdata

# Create a custom prompt template for the RAG graph
# This prompt allows for interpretation and reasoning while staying grounded in the documents
prompt_template = """You are an expert assistant helping to answer questions based on the provided context from documents.\n\nYour task is to:\n1. Use the context as the foundation for your answer\n2. If the context doesn't contain enough information, clearly state what is missing\n\nContext from documents:\n{context}\n\nQuestion: {question}\n\nInstructions:\n- Base your answer on the provided context, but feel free to interpret, explain, and make logical connections\n- If the context is insufficient, say you cannot answer the question.\n\nProvide a thoughtful, well-reasoned answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

class RAGState(TypedDict):
    question: str
    context: List[Document]
    answer: str

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

def retrieve(state: RAGState) -> dict:
    """Retrieve relevant documents based on the question."""
    docs = retriever.invoke(state["question"])
    return {"context": docs}

def generate(state: RAGState) -> dict:
    """Format context, prompt the LLM, and return the answer."""
    formatted_context = "\n\n".join([
        f"[From {doc.metadata.get('source', 'Unknown')}]\n{doc.page_content}"
        for doc in state["context"]
    ])
    prompt = PROMPT.format(context=formatted_context, question=state["question"])
    response = llm.invoke(prompt)
    return {"answer": response.content}

workflow = StateGraph(RAGState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

rag_graph = workflow.compile()

print("LangGraph RAG pipeline created and ready to use")

LangGraph RAG pipeline created and ready to use


## Step 6: Test Retrieval

This step validates that the retrieval system is working correctly before generating answers:

- **Define test query**: Create a sample question to evaluate the retrieval functionality
- **Execute retrieval**: Query the vector store to find the most semantically similar document chunks
- **Display results**: Show the retrieved documents with their source files and content previews
- **Verify relevance**: Check that the retrieved chunks are actually relevant to the test query
- **Confirm functionality**: Ensure the retrieval system is finding appropriate context for questions

Follow along with the workshop and try out different test queries based off the data in the folder!

In [ ]:
# Test retrieval to see what documents are found
test_query = "What are Monte Carlo methods?"
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Use invoke for newer LangChain versions, fallback to get_relevant_documents
try:
    retrieved_docs = retriever.invoke(test_query)
except AttributeError:
    retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"Retrieved {len(retrieved_docs)} documents for query: '{test_query}'\n")
for i, doc in enumerate(retrieved_docs, 1):
    source = doc.metadata.get('source', 'Unknown')
    print(f"{i}. From: {source}")
    print(f"   Content preview: {doc.page_content[:500]}...\n")


Retrieved 3 documents for query: 'What are Monte Carlo methods?'

1. From: Probability and statistics for computer scientists.pdf
   Content preview: complicated, risky, consuming, expensive, or impossible. For example, suppose a complex
device or machine is to be built and launched. Before it happens, its performance is sim-
ulated, and this allows experts to evaluate its adequacy and associated risks carefully and
safely. For example, one surely prefers to evaluate reliability and safety of a new module of
a space station by means of computer simulations rather than during the actual mission.
Monte Carlo methods are mostly used for the comp...

2. From: Probability and statistics for computer scientists.pdf
   Content preview: 104 Probability and Statistics for Computer Scientists
FIGURE 5.1:Casino Monte Carlo in Principality
of Monaco.
Monte Carlo methods and Monte Carlo
studies inherit their name from Eu-
rope’s most famousMonte Carlo casino
(Figure 5.1) located in Principality of


## Step 7: RAG Query Function

This step creates a user-friendly function to query the RAG system:

- **Define query function**: Create a `rag_query()` function that handles the complete RAG workflow
- **Retrieve context**: Automatically fetch relevant document chunks based on the user's question
- **Generate answer**: Use the LLM to synthesize an answer from the retrieved context
- **Display results**: Present the question, answer, and optionally the source documents used
- **Format output**: Organize the response in a readable format with clear separation between question, answer, and sources
- **Test functionality**: Run a sample query to demonstrate the complete RAG system in action

Follow along with the workshop and try out different test queries based off the data in the folder!

In [ ]:
def rag_query(query: str, show_sources: bool = True):
    """
    Perform a RAG query: retrieve relevant context and generate answer using Gemini.

    Uses the LangGraph RAG pipeline to retrieve documents and generate an answer.

    Args:
        query: The user's question
        show_sources: Whether to show source documents

    Returns:
        Dictionary with 'answer' and 'context'
    """
    result = rag_graph.invoke({"question": query})

    if show_sources:
        print(f"Question: {query}\n")
        print(f"Answer:\n{result['answer']}\n")
        print("=" * 80)
        print("Source Documents:")
        for i, doc in enumerate(result['context'], 1):
            source = doc.metadata.get('source', 'Unknown')
            print(f"\n{i}. Source: {source}")
            print(f"   Preview: {doc.page_content[:150]}...")
    else:
        print(f"Q: {query}\n")
        print(f"A: {result['answer']}\n")

    return result

# Test RAG query
test_question = "What are the key points discussed in the documents?"
rag_query(test_question)


AttributeError: 'str' object has no attribute 'content'

## Step 8: Interactive Query Interface

This step creates an interactive command-line interface for continuous querying:

- **Create interactive loop**: Build a function that allows users to ask multiple questions in a session
- **Handle user input**: Accept questions from the user via command-line input
- **Process queries**: Execute RAG queries for each user question and display results
- **Manage session**: Provide commands to exit the interactive mode (quit/exit/q)
- **Error handling**: Catch and display any errors that occur during query processing
- **User experience**: Provide clear instructions and formatting for a smooth interaction experience

Run the cell, ask questions, and quit when done!

In [ ]:
def interactive_rag():
    """
    Interactive function to ask questions and get RAG-based answers.
    """
    print("RAG System Ready! Ask questions about the documents.")
    print("Type 'quit' or 'exit' to stop.\n")

    while True:
        query = input("Your question: ").strip()

        if query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break

        if not query:
            continue

        try:
            result = rag_query(query, show_sources=True)
            print("-" * 80 + "\n")
        except Exception as e:
            print(f"Error: {e}\n")


interactive_rag()
